In [24]:
#Apoorva Ambulgekar 60004180007 AIML A1 TE Comps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import  train_test_split
from sklearn import metrics
import seaborn as sns
from numpy import log2 as log
import pprint
import math


In [25]:
eps=np.finfo(float).eps
df=pd.read_csv("dataset_covid19.csv")

In [26]:
df

,ID,Fever,Cough,Breathing_issues,Infected,Unnamed: 5,Unnamed: 6
0,1,No,No,No,No,NaN,NaN
1,2,Yes,Yes,Yes,Yes,NaN,NaN
2,3,Yes,Yes,No,No,NaN,NaN
3,4,Yes,No,Yes,Yes,NaN,NaN
4,5,Yes,Yes,Yes,Yes,NaN,NaN
5,6,No,Yes,No,No,NaN,NaN
6,7,Yes,No,Yes,Yes,NaN,NaN
7,8,Yes,No,Yes,Yes,NaN,NaN
8,9,No,Yes,Yes,Yes,NaN,NaN
9,10,Yes,Yes,No,Yes,NaN,NaN


In [27]:
df.columns

Index(['ID', 'Fever', 'Cough', 'Breathing_issues', 'Infected', 'Unnamed: 5',
       'Unnamed: 6'],
      dtype='object')

In [28]:
df=df.drop(['Unnamed: 5','Unnamed: 6'],axis=1)

In [29]:
df

,ID,Fever,Cough,Breathing_issues,Infected
0,1,No,No,No,No
1,2,Yes,Yes,Yes,Yes
2,3,Yes,Yes,No,No
3,4,Yes,No,Yes,Yes
4,5,Yes,Yes,Yes,Yes
5,6,No,Yes,No,No
6,7,Yes,No,Yes,Yes
7,8,Yes,No,Yes,Yes
8,9,No,Yes,Yes,Yes
9,10,Yes,Yes,No,Yes


In [30]:
df=df.drop(['ID'],axis=1)

In [31]:
df

,Fever,Cough,Breathing_issues,Infected
0,No,No,No,No
1,Yes,Yes,Yes,Yes
2,Yes,Yes,No,No
3,Yes,No,Yes,Yes
4,Yes,Yes,Yes,Yes
5,No,Yes,No,No
6,Yes,No,Yes,Yes
7,Yes,No,Yes,Yes
8,No,Yes,Yes,Yes
9,Yes,Yes,No,Yes


In [32]:
Entropy=(-(8/14)*math.log(8/14)/math.log(2))-((6/14)*math.log(6/14)/math.log(2))
Entropy

0.9852281360342516

In [33]:
y=df.keys()[-1]
x=df[y].unique()
x

array(['No', 'Yes'], dtype=object)

In [34]:
def find_entropy(df):
    Class=df.keys()[-1]
    entropy=0
    values=df[Class].unique()
    for value in values:
        fraction=df[Class].value_counts()[value]/len(df[Class])
        entropy+=-fraction*np.log2(fraction)
    return entropy

In [35]:
def find_entropy_attribute(df,attribute):
    Class=df.keys()[-1]
    target_variables=df[Class].unique()
    variables =df[attribute].unique()
    entropy2=0
    for variable in variables:
        entropy=0
        for target_variable in target_variables:
            num=len(df[attribute][df[attribute]==variable][df[Class]==target_variable])
            den=len(df[attribute][df[attribute]==variable])
            fraction=num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2=den/len(df)
        entropy2 += fraction2*entropy
    return abs(entropy2)

In [36]:
def find_winner(df):
    Entropy_att=[]
    IG=[]
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]


In [37]:
def get_subtable(df,node,value):
    return df[df[node]==value].reset_index(drop=True)

In [38]:
def buildTree(df,tree=None):
    Class=df.keys()[-1]
    node=find_winner(df)
    attValue=np.unique(df[node])
    if tree is None:
        tree={}
        tree[node]={}
    for value in attValue:
        subtable=get_subtable(df,node,value)
        clvalue,counts=np.unique(subtable['Fever'],return_counts=True)
        if len(counts)==1:
            tree[node][value]=clvalue[0]
        else:
            tree[node][value]=buildTree(subtable)
    return tree

In [39]:
t=buildTree(df,tree=None)
pprint.pprint(t)

{'Breathing_issues': {'No': {'Fever': {'No': 'No', 'Yes': 'Yes'}},
                      'Yes': {'Fever': {'No': 'No', 'Yes': 'Yes'}}}}


In [40]:
df.head()

,Fever,Cough,Breathing_issues,Infected
0,No,No,No,No
1,Yes,Yes,Yes,Yes
2,Yes,Yes,No,No
3,Yes,No,Yes,Yes
4,Yes,Yes,Yes,Yes


In [41]:
t

{'Breathing_issues': {'No': {'Fever': {'No': 'No', 'Yes': 'Yes'}},
  'Yes': {'Fever': {'No': 'No', 'Yes': 'Yes'}}}}